**This notebook serves to demonstrate how to integrate the `auth.mast` service into a Django web application.  The code was adopted from the solution derived for the `jwql` web application.  Much of the code was written by Christian Mesh of the Archive Science Branch.  Thank you Christian!**

**Please note that this notebook merely serves as a reference, and does not work right out of the box -- the cells within are not meant to be executed, and executing most of them would result in an error.  There are various tokens and metadata that must be provided, and the code within is designed for a specific project.  It may be beneficial to structure the code in a different way for different applications.  This notebook simply serves as a guide.**

**Resources:**
- The `auth.mast` service can be found here: https://auth.mast.stsci.edu/
- The development version of the `auth.mast` service can be found here:  https://auth.mastdev.stsci.edu/


**In this bare-bones example, we assume the following directory structure for the `django` web application:**

```
website/   
    apps/
        my_app/
            views.py
            urls.py
            templates/
                some_webpage.html
```

**Required third-party libraries**

- `authlib`
- `django`

In [ ]:
# Necessary imports
import requests

from authlib.django.client import OAuth
from django.shortcuts import redirect, render
from django.urls import path

In [ ]:
# Set some necessary global parameters
client_id = ''  # Provided by ASB
client_secret = ''  # Provided by ASB
auth_mast = 'auth.mast.stsci.edu'  # or can use authdev.mast.stsci.edu for dev
base_url = 'https://myapp.stsci.edu'  # whatever URL your application uses

*Note that the `client_id` and `client_secret` are tokens that are specific to the application.  To obtain these tokens, please contact Christian Mesh (cmesh@stsci.edu) or someone else from the ASB branch*

### Registering the application with `auth.mast`

*The following function should be placed within the scope of the `views.py` module*

In [ ]:
def register_oauth():
    """Register the ``my_app`` application with the ``auth.mast``
    authentication service.

    Returns
    -------
    oauth : Object
        An object containing methods to authenticate a user, provided
        by the ``auth.mast`` service.
    """

    oauth = OAuth()
    client_kwargs = {'scope': 'mast:user:info'}
    oauth.register(
        'mast_auth',
        client_id='{}'.format(client_id),
        client_secret='{}'.format(client_secret),
        access_token_url='https://{}/oauth/access_token?client_secret={}'.format(auth_mast, client_secret),
        access_token_params=None,
        refresh_token_url=None,
        authorize_url='https://{}/oauth/authorize'.format(auth_mast),
        api_base_url='https://{}/1.1/'.format(auth_mast),
        client_kwargs=client_kwargs)

    return oauth

In [ ]:
MY_APP_OAUTH = register_oauth()

### Convenience Decorators

*The following decorators can be used to easily pass authenticaion metadata to various views within your web application, as will be demonstrated below.  These decorators should be placed within the scope of `views.py`*

In [ ]:
def auth_info(fn):
    """A decorator function that will return user metadata from the
    authentication along with what is returned by the original function.

    Parameters
    ----------
    fn : function
        The function to decorate

    Returns
    -------
    user_info : function
        The decorated function
    """

    def user_info(request, **kwargs):
        """Store authenticated user credentials in a cookie and return
        it.  If the user is not authenticated, store no credentials in
        the cookie.

        Parameters
        ----------
        request : HttpRequest object
            Incoming request from the webpage

        Returns
        -------
        fn : function
            The decorated function
        """

        cookie = request.COOKIES.get("ASB-AUTH")

        # If user is authenticated, return user credentials
        if cookie is not None:
            response = requests.get(
                'https://{}/info'.format(auth_mast),
                headers={'Accept': 'application/json',
                         'Authorization': 'token {}'.format(cookie)})
            response = response.json()

        # If user is not authenticated, return no credentials
        else:
            response = {'ezid' : None, "anon": True}

        return fn(request, response, **kwargs)

    return user_info

In [ ]:
def auth_required(fn):
    """A decorator function that requires the passed function to have
    authentication through ``auth.mast`` set up.

    Parameters
    ----------
    fn : function
        The function to decorate

    Returns
    -------
    check_auth : function
        The decorated function
    """

    @auth_info
    def check_auth(request, user):
        """Check if the user is authenticated through ``auth.mast``.
        If not, perform the authorization.

        Parameters
        ----------
        request : HttpRequest object
            Incoming request from the webpage
        user : dict
            A dictionary of user credentials

        Returns
        -------
        fn : function
            The decorated function
        """

        # If user is currently anonymous, require a login
        if user["anon"]:
            # Redirect to oauth login
            redirect_uri = os.path.join(base_url, 'authorize')
            return MY_APP_OAUTH.mast_auth.authorize_redirect(request, redirect_uri)

        return fn(request, user)

    return check_auth

### Authentication-related URLs

*The following URLs serve as the URLs that point to individual authentication-related views within the `django` web application (e.g. `login`, `logout`, etc).  They should be placed in `apps.my_app.urls.py`.*

In [ ]:
# Your app probably has some existing URLs:
urlpatterns = [
    path('', views.home, name='home'),
    path('about/', views.about, name='about'),
]

# These are the authenticaion related views we will build
auth_urlpatterns = [
    path('login/', views.login, name='login'),
    path('logout/', views.logout, name='logout'),
    path('authorize/', views.authorize, name='authorize'),
]

# Add these to the existing URL patterns
urlpatterns += auth_urlpatters

### Authentication-related Views

*The following functions serve as individual authentication-related views within the `django` web application.  They should be placed within the scope of `views.py`*

In [ ]:
def authorize(request):
    """Spawn the authentication process for the user

    The authentication process involves retreiving an access token
    from ``auth.mast`` and porting the data to a cookie.

    Parameters
    ----------
    request : HttpRequest object
        Incoming request from the webpage

    Returns
    -------
    HttpResponse object
        Outgoing response sent to the webpage
    """

    # Get auth.mast token
    token = MY_APP_OAUTH.mast_auth.authorize_access_token(request, headers={'Accept': 'application/json'})

    # Set secure cookie parameters
    cookie_args = {}
    cookie_args['domain'] = 'myapp.stsci.edu'
    cookie_args['secure'] = True  
    cookie_args['httponly'] = True

    # Set the cookie and redirect to home
    response = redirect("/")
    response.set_cookie("ASB-AUTH", token["access_token"], **cookie_args)

    return response

In [ ]:
@auth_required
def login(request, user):
    """Spawn a login process for the user

    The ``auth_requred`` decorator is used to require that the user
    authenticate through ``auth.mast``, then the user is redirected
    back to the homepage.

    Parameters
    ----------
    request : HttpRequest object
        Incoming request from the webpage
    user : dict
        A dictionary of user credentials.

    Returns
    -------
    HttpResponse object
        Outgoing response sent to the webpage
    """

    return redirect("/")

In [ ]:
def logout(request):
    """Spawn a logout process for the user

    Upon logout, the user's ``auth.mast`` credientials are removed and
    the user is redirected back to the homepage.

    Parameters
    ----------
    request : HttpRequest object
        Incoming request from the webpage
    user : dict
        A dictionary of user credentials.

    Returns
    -------
    HttpResponse object
        Outgoing response sent to the webpage
    """

    response = redirect("/")
    response.delete_cookie("ASB-AUTH")

    return response


### Accessing authentication metadata within other views

*The following demonstrates how to use the `@auth_info` decorator on an example view in order to utilize authentication metadata.  In this example, we will consider "some_webpage" in which we only want to show content if the user is authenticated.  We assume that this function is placed within the scope of `views.py`*

In [ ]:
@auth_info
def some_webpage(request, user):
    """Generate the ``some_webpage`` page.

    Parameters
    ----------
    request : HttpRequest object
        Incoming request from the webpage

    Returns
    -------
    HttpResponse object
        Outgoing response sent to the webpage
    """
    template = 'some_webpage.html'
    context = {'user': user}

    return render(request, template, context)

*The `some_webpage.html` file now has access to the `user` data, which is the cookie provided by `auth.mast`.  Here is an example of what a `div` in `some_webpage.html` might look like; it provides a login/logout button, and displays the user's `ezid`.  In this example, we use the `jinja2` templating language to access `user`, however, one might use the built in `django` templating language.*

```html
<div>
    {% if user.ezid %}
        <p>{{ user.ezid }}</p>
        <a role="button" class="btn btn-primary" href='https://myapp.stsci.edu/logout'>logout</a>
    {% else %}
        <a role="button" class="btn btn-primary" href='https://myapp.stsci.edu/login'>login</a>
    {% endif %}
</div>
```

*To see which information is contained within the `user` cookie, visit https://auth.mastdev.stsci.edu/info*